# LeetCode SQL Problems

### Table of Contents
- [Problem 1757: Recyclable and Low Fat Products](#Problem-1757-Recyclable-and-Low-Fat-Products) EASY, SELECT
- [Problem 584: Find Customer Referee](#Problem-584-Find-Customer-Referee) EASY, SELECT, HandleNull
- [Problem 595: Big Countries](#Problem-595-Big-Countries) EASY, SELECT
- [Problem 1148: Article Views I](#Problem-1148-Article-Views-I) EASY, SELECT
- [Problem 1683: Invalid Tweets](#problem-1683-nvalid-tweets) EASY,SELECT
-  Problem 1378. Replace Employee ID With The Unique Identifier.  EASY, JOIN
-  Problem 1068. Product Sales Analysis I. EASY, JOIN
-  [1581. Customer Who Visited but Did Not Make Any Transactions](https://leetcode.com/problems/customer-who-visited-but-did-not-make-any-transactions/?envType=study-plan-v2&envId=top-sql-50) EASY*, JOIN


In [ ]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import functions as F, Window as W


In [ ]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('LeetCode Problems') \
    .getOrCreate()

In [70]:
# download Taxi trip yellow data and zones
# !wget https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-09.parquet -P data/yellow_taxi/
# !wget https://d37ci6vzurychx.cloudfront.net/misc/taxi_zone_lookup.csv -P data/yellow_taxi/

--2024-12-14 21:02:39--  https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-09.parquet
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 3.166.97.40, 3.166.97.18, 3.166.97.164, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|3.166.97.40|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 61170186 (58M) [binary/octet-stream]
Saving to: ‘data/yellow_taxi/yellow_tripdata_2024-09.parquet’

yellow_tripdata_202 100%[===================>]  58.34M  8.05MB/s    in 7.7s    

2024-12-14 21:02:47 (7.62 MB/s) - ‘data/yellow_taxi/yellow_tripdata_2024-09.parquet’ saved [61170186/61170186]

--2024-12-14 21:02:47--  https://d37ci6vzurychx.cloudfront.net/misc/taxi_zone_lookup.csv
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 3.166.97.40, 3.166.97.18, 3.166.97.164, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|3.166.97.40|:443... connected.
HTTP r

In [ ]:
# Load the Yellow Taxi Trip Data (Parquet)
yellow_taxi_df = spark.read.parquet("data/yellow_taxi/yellow_tripdata_2024-09.parquet")

# Load the Taxi Zone Lookup Data (CSV)
taxi_zone_df = spark.read.csv("data/yellow_taxi/taxi_zone_lookup.csv", header=True, inferSchema=True)


### Problem 1757: Recyclable and Low Fat Products

```md
+-------------+---------+
| Column Name | Type    |
+-------------+---------+
| product_id  | int     |
| low_fats    | enum    |
| recyclable  | enum    |
+-------------+---------+
product_id is the primary key (column with unique values) for this table.
low_fats is an ENUM (category) of type ('Y', 'N') where 'Y' means this product is low fat and 'N' means it is not.
recyclable is an ENUM (category) of types ('Y', 'N') where 'Y' means this product is recyclable and 'N' means it is not.

 

Write a solution to find the ids of products that are both low fat and recyclable.

Return the result table in any order.

The result format is in the following example.

 

Example 1:

Input: 
Products table:
+-------------+----------+------------+
| product_id  | low_fats | recyclable |
+-------------+----------+------------+
| 0           | Y        | N          |
| 1           | Y        | Y          |
| 2           | N        | Y          |
| 3           | Y        | Y          |
| 4           | N        | N          |
+-------------+----------+------------+
Output: 
+-------------+
| product_id  |
+-------------+
| 1           |
| 3           |
+-------------+
Explanation: Only products 1 and 3 are both low fat and recyclable.
```

MS SQL Server 

```sql
SELECT product_id
From Products 
WHERE low_fats = 'Y' and recyclable = 'Y';
```

Pyspark

In [3]:
data = [
    (0, 'Y', 'N'),
    (1, 'Y', 'Y'),
    (2, 'N', 'Y'),
    (3, 'Y', 'Y'),
    (4, 'N', 'N'),
    (5, 'Y', 'N'),
    (6, 'N', 'Y'),
    (7, 'Y', 'Y'),
    (8, 'N', 'N'),
    (9, 'Y', 'Y'),
    (10, 'N', 'N')
]



columns = ['product_id', 'low_fats', 'recyclable']

df = spark.createDataFrame(data, columns)

```
+-------------+
| product_id  |
+-------------+
| 1           |
| 3           |
| 7           |
| 9           |
+-------------+
```

In [14]:
result = (
    df
    .filter((F.col('low_fats') == 'Y') & (F.col('recyclable') == 'Y'))
    .select('product_id')
)

In [16]:
result.show()

+----------+
|product_id|
+----------+
|         1|
|         3|
|         7|
|         9|
+----------+



### Problem 584: Find Customer Referee

```
Table: Customer

+-------------+---------+
| Column Name | Type    |
+-------------+---------+
| id          | int     |
| name        | varchar |
| referee_id  | int     |
+-------------+---------+
In SQL, id is the primary key column for this table.
Each row of this table indicates the id of a customer, their name, and the id of the customer who referred them.

 

Find the names of the customer that are not referred by the customer with id = 2.

Return the result table in any order.

The result format is in the following example.

 

Example 1:

Input: 
Customer table:
+----+------+------------+
| id | name | referee_id |
+----+------+------------+
| 1  | Will | null       |
| 2  | Jane | null       |
| 3  | Alex | 2          |
| 4  | Bill | null       |
| 5  | Zack | 1          |
| 6  | Mark | 2          |
+----+------+------------+
Output: 
+------+
| name |
+------+
| Will |
| Jane |
| Bill |
| Zack |
+------+
```

```sql
SELECT [name]
FROM Customer
where COALESCE(referee_id,0) != 2;
```

In [19]:
data = [
    (1, "Will", None),
    (2, "Jane", None),
    (3, "Alex", 2),
    (4, "Bill", None),
    (5, "Zack", 1),
    (6, "Mark", 2),
    (7, "Emma", 3),
    (8, "John", None),
    (9, "Lucy", 4),
    (10, "David", 2),
    (11, "Sophie", 5),
    (12, "Adam", None)
]

columns = ["id", "name", "referee_id"]

df = spark.createDataFrame(data, columns)
df.show()

+---+------+----------+
| id|  name|referee_id|
+---+------+----------+
|  1|  Will|      NULL|
|  2|  Jane|      NULL|
|  3|  Alex|         2|
|  4|  Bill|      NULL|
|  5|  Zack|         1|
|  6|  Mark|         2|
|  7|  Emma|         3|
|  8|  John|      NULL|
|  9|  Lucy|         4|
| 10| David|         2|
| 11|Sophie|         5|
| 12|  Adam|      NULL|
+---+------+----------+



In [35]:
result = (
    df
    .filter((F.col('referee_id') != 2) | (F.col("referee_id").isNull()))
    .select('name')
)

In [36]:
result.show()

+------+
|  name|
+------+
|  Will|
|  Jane|
|  Bill|
|  Zack|
|  Emma|
|  John|
|  Lucy|
|Sophie|
|  Adam|
+------+



### Problem 595: Big Countries

```
Table: World

+-------------+---------+
| Column Name | Type    |
+-------------+---------+
| name        | varchar |
| continent   | varchar |
| area        | int     |
| population  | int     |
| gdp         | bigint  |
+-------------+---------+
name is the primary key (column with unique values) for this table.
Each row of this table gives information about the name of a country, the continent to which it belongs, its area, the population, and its GDP value.

 

A country is big if:

    it has an area of at least three million (i.e., 3000000 km2), or
    it has a population of at least twenty-five million (i.e., 25000000).

Write a solution to find the name, population, and area of the big countries.

Return the result table in any order.

The result format is in the following example.

 

Example 1:

Input: 
World table:
+-------------+-----------+---------+------------+--------------+
| name        | continent | area    | population | gdp          |
+-------------+-----------+---------+------------+--------------+
| Afghanistan | Asia      | 652230  | 25500100   | 20343000000  |
| Albania     | Europe    | 28748   | 2831741    | 12960000000  |
| Algeria     | Africa    | 2381741 | 37100000   | 188681000000 |
| Andorra     | Europe    | 468     | 78115      | 3712000000   |
| Angola      | Africa    | 1246700 | 20609294   | 100990000000 |
+-------------+-----------+---------+------------+--------------+
Output: 
+-------------+------------+---------+
| name        | population | area    |
+-------------+------------+---------+
| Afghanistan | 25500100   | 652230  |
| Algeria     | 37100000   | 2381741 |
+-------------+------------+---------+

```

```sql
SELECT [name], [population], [area]
from World
where [area] >= 3000000 OR [population] >= 25000000;
```

In [37]:
data = [
    ("Afghanistan", "Asia", 652230, 25500100, 20343000000),
    ("Albania", "Europe", 28748, 2831741, 12960000000),
    ("Algeria", "Africa", 2381741, 37100000, 188681000000),
    ("Andorra", "Europe", 468, 78115, 3712000000),
    ("Angola", "Africa", 1246700, 20609294, 100990000000)
]

columns = ["name", "continent", "area", "population", "gdp"]

# Create DataFrame
df = spark.createDataFrame(data, columns)

In [40]:
result = (
    df
    .filter((F.col('population') > 25000000) | (F.col('area') > 3000000))
    .select('name','population', 'area')
)

In [41]:
result.show()

+-----------+----------+-------+
|       name|population|   area|
+-----------+----------+-------+
|Afghanistan|  25500100| 652230|
|    Algeria|  37100000|2381741|
+-----------+----------+-------+



### Problem 1148: Article Views I

```
Table: Views

+---------------+---------+
| Column Name   | Type    |
+---------------+---------+
| article_id    | int     |
| author_id     | int     |
| viewer_id     | int     |
| view_date     | date    |
+---------------+---------+
There is no primary key (column with unique values) for this table, the table may have duplicate rows.
Each row of this table indicates that some viewer viewed an article (written by some author) on some date. 
Note that equal author_id and viewer_id indicate the same person.

 

Write a solution to find all the authors that viewed at least one of their own articles.

Return the result table sorted by id in ascending order.

The result format is in the following example.

 

Example 1:

Input: 
Views table:
+------------+-----------+-----------+------------+
| article_id | author_id | viewer_id | view_date  |
+------------+-----------+-----------+------------+
| 1          | 3         | 5         | 2019-08-01 |
| 1          | 3         | 6         | 2019-08-02 |
| 2          | 7         | 7         | 2019-08-01 |
| 2          | 7         | 6         | 2019-08-02 |
| 4          | 7         | 1         | 2019-07-22 |
| 3          | 4         | 4         | 2019-07-21 |
| 3          | 4         | 4         | 2019-07-21 |
+------------+-----------+-----------+------------+
Output: 
+------+
| id   |
+------+
| 4    |
| 7    |
+------+

```

```sql
select DISTINCT author_id as id
from Views
where author_id = viewer_id 
ORDER BY author_id;
```


In [42]:
data = [
    (1, 3, 5, '2019-08-01'),
    (1, 3, 6, '2019-08-02'),
    (2, 7, 7, '2019-08-01'),
    (2, 7, 6, '2019-08-02'),
    (4, 7, 1, '2019-07-22'),
    (3, 4, 4, '2019-07-21'),
    (3, 4, 4, '2019-07-21')
]

columns = ["article_id", "author_id", "viewer_id", "view_date"]

# Create DataFrame
df = spark.createDataFrame(data, columns)

In [52]:
result = (
    df
    .filter(F.col('author_id') == F.col('viewer_id'))
    .select('author_id')
    .distinct()
    .orderBy('author_id')
)

In [53]:
result.show()

+---------+
|author_id|
+---------+
|        4|
|        7|
+---------+



### Problem 1683: Invalid Tweets

```
+----------------+---------+
| Column Name    | Type    |
+----------------+---------+
| tweet_id       | int     |
| content        | varchar |
+----------------+---------+
tweet_id is the primary key (column with unique values) for this table.
content consists of characters on an American Keyboard, and no other special characters.
This table contains all the tweets in a social media app.

 

Write a solution to find the IDs of the invalid tweets. The tweet is invalid if the number of characters used in the content of the tweet is strictly greater than 15.

Return the result table in any order.

The result format is in the following example.

 

Example 1:

Input: 
Tweets table:
+----------+-----------------------------------+
| tweet_id | content                           |
+----------+-----------------------------------+
| 1        | Let us Code                       |
| 2        | More than fifteen chars are here! |
+----------+-----------------------------------+
Output: 
+----------+
| tweet_id |
+----------+
| 2        |
+----------+
Explanation: 
Tweet 1 has length = 11. It is a valid tweet.
Tweet 2 has length = 33. It is an invalid tweet.

```


```sql
SELECT tweet_id
from Tweets
where LEN(content) > 15
```

In [56]:
data = [
    (1, 'Let us Code'),
    (2, 'More than fifteen chars are here!')
]

columns = ["tweet_id", "content"]
df = spark.createDataFrame(data, columns)

In [58]:
result = (
    df
    .filter(F.length(F.col("content")) > 15)
    .select("tweet_id")
)

In [59]:
result.show()

+--------+
|tweet_id|
+--------+
|       2|
+--------+



### Problem 1378. Replace Employee ID With The Unique Identifier



```
Table: Employees

+---------------+---------+
| Column Name   | Type    |
+---------------+---------+
| id            | int     |
| name          | varchar |
+---------------+---------+
id is the primary key (column with unique values) for this table.
Each row of this table contains the id and the name of an employee in a company.

 

Table: EmployeeUNI

+---------------+---------+
| Column Name   | Type    |
+---------------+---------+
| id            | int     |
| unique_id     | int     |
+---------------+---------+
(id, unique_id) is the primary key (combination of columns with unique values) for this table.
Each row of this table contains the id and the corresponding unique id of an employee in the company.

 

Write a solution to show the unique ID of each user, If a user does not have a unique ID replace just show null.

Return the result table in any order.

The result format is in the following example.

 

Example 1:

Input: 
Employees table:
+----+----------+
| id | name     |
+----+----------+
| 1  | Alice    |
| 7  | Bob      |
| 11 | Meir     |
| 90 | Winston  |
| 3  | Jonathan |
+----+----------+
EmployeeUNI table:
+----+-----------+
| id | unique_id |
+----+-----------+
| 3  | 1         |
| 11 | 2         |
| 90 | 3         |
+----+-----------+
Output: 
+-----------+----------+
| unique_id | name     |
+-----------+----------+
| null      | Alice    |
| null      | Bob      |
| 2         | Meir     |
| 3         | Winston  |
| 1         | Jonathan |
+-----------+----------+
Explanation: 
Alice and Bob do not have a unique ID, We will show null instead.
The unique ID of Meir is 2.
The unique ID of Winston is 3.
The unique ID of Jonathan is 1.

```

```sql
select EUI.unique_id, EMP.[name]
from Employees as EMP
left join EmployeeUNI as EUI on EMP.id = EUI.id; 
```

In [60]:
employees_data = [(1, 'Alice'),
                  (7, 'Bob'),
                  (11, 'Meir'),
                  (90, 'Winston'),
                  (3, 'Jonathan')]
employeeuni_data = [(3, 1),
                    (11, 2),
                    (90, 3)]



employees_df = spark.createDataFrame(employees_data, ['id', 'name'])
employeeuni_df = spark.createDataFrame(employeeuni_data, ['id', 'unique_id'])

In [64]:

result = employees_df.join(employeeuni_df, on='id', how='left') \
                        .select('unique_id', 'name') \
                        .orderBy('name')

In [65]:
result.show()

+---------+--------+
|unique_id|    name|
+---------+--------+
|     NULL|   Alice|
|     NULL|     Bob|
|        1|Jonathan|
|        2|    Meir|
|        3| Winston|
+---------+--------+



Note: If `employees_df` is significantly larger than `employeeuni_df`, then broadcast `employeeuni_df` to all worker nodes to minimize the costly shuffle operation and optimize the join performance.

 pyspark.sql.functions.broadcast(df)[source]

    Marks a DataFrame as small enough for use in broadcast joins.


In [67]:
# Broadcast the smaller DataFrame (employeeuni_df)
result = employees_df.join(F.broadcast(employeeuni_df), on='id', how='left') \
                        .select('unique_id', 'name') \
                        .orderBy('name')

In [68]:
result.show()

+---------+--------+
|unique_id|    name|
+---------+--------+
|     NULL|   Alice|
|     NULL|     Bob|
|        1|Jonathan|
|        2|    Meir|
|        3| Winston|
+---------+--------+



In [87]:
yellow_taxi_df.printSchema()
taxi_zone_df.printSchema()

root
 |-- VendorID: integer (nullable = true)
 |-- tpep_pickup_datetime: timestamp_ntz (nullable = true)
 |-- tpep_dropoff_datetime: timestamp_ntz (nullable = true)
 |-- passenger_count: long (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: long (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- payment_type: long (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- Airport_fee: double (nullable = true)

root
 |-- LocationID: integer (nullable = true)
 |-- Borough: string (nullable = true)
 |-- Zone: string (nullable = true)
 |

In [102]:
result = yellow_taxi_df.join(taxi_zone_df, yellow_taxi_df.PULocationID == taxi_zone_df.LocationID, "left") \
    .select("PULocationID", "DOLocationID", "Trip_distance", "Total_amount", "Zone","Borough") \
    .orderBy("tpep_pickup_datetime")


In [103]:
result.limit(5).toPandas()

,PULocationID,DOLocationID,Trip_distance,Total_amount,Zone,Borough
0,163,237,0.71,12.20,Midtown North,Manhattan
1,132,100,17.86,105.23,JFK Airport,Queens
2,68,230,2.05,33.25,East Chelsea,Manhattan
3,141,235,7.77,47.40,Lenox Hill West,Manhattan
4,166,237,3.58,32.28,Morningside Heights,Manhattan


### Problem 1068. Product Sales Analysis I


```
Table: Sales

+-------------+-------+
| Column Name | Type  |
+-------------+-------+
| sale_id     | int   |
| product_id  | int   |
| year        | int   |
| quantity    | int   |
| price       | int   |
+-------------+-------+
(sale_id, year) is the primary key (combination of columns with unique values) of this table.
product_id is a foreign key (reference column) to Product table.
Each row of this table shows a sale on the product product_id in a certain year.
Note that the price is per unit.

 

Table: Product

+--------------+---------+
| Column Name  | Type    |
+--------------+---------+
| product_id   | int     |
| product_name | varchar |
+--------------+---------+
product_id is the primary key (column with unique values) of this table.
Each row of this table indicates the product name of each product.

 

Write a solution to report the product_name, year, and price for each sale_id in the Sales table.

Return the resulting table in any order.

The result format is in the following example.

 

Example 1:

Input: 
Sales table:
+---------+------------+------+----------+-------+
| sale_id | product_id | year | quantity | price |
+---------+------------+------+----------+-------+ 
| 1       | 100        | 2008 | 10       | 5000  |
| 2       | 100        | 2009 | 12       | 5000  |
| 7       | 200        | 2011 | 15       | 9000  |
+---------+------------+------+----------+-------+
Product table:
+------------+--------------+
| product_id | product_name |
+------------+--------------+
| 100        | Nokia        |
| 200        | Apple        |
| 300        | Samsung      |
+------------+--------------+
Output: 
+--------------+-------+-------+
| product_name | year  | price |
+--------------+-------+-------+
| Nokia        | 2008  | 5000  |
| Nokia        | 2009  | 5000  |
| Apple        | 2011  | 9000  |
+--------------+-------+-------+
Explanation: 
From sale_id = 1, we can conclude that Nokia was sold for 5000 in the year 2008.
From sale_id = 2, we can conclude that Nokia was sold for 5000 in the year 2009.
From sale_id = 7, we can conclude that Apple was sold for 9000 in the year 2011.

```

```sql
Select P.product_name, S.[year], S.price
from Product as P
inner join Sales as S 
on S.product_id = P.product_id
```

In [104]:

spark = SparkSession.builder.appName("SalesProductJoin").getOrCreate()


sales_data = [
    (1, 100, 2008, 10, 5000),
    (2, 100, 2009, 12, 5000),
    (7, 200, 2011, 15, 9000)
]
sales_columns = ["sale_id", "product_id", "year", "quantity", "price"]
sales_df = spark.createDataFrame(sales_data, sales_columns)


product_data = [
    (100, "Nokia"),
    (200, "Apple"),
    (300, "Samsung")
]
product_columns = ["product_id", "product_name"]
product_df = spark.createDataFrame(product_data, product_columns)


result = sales_df.join(product_df, on="product_id", how="inner") \
                    .select("product_name", "year", "price")

result.show()

+------------+----+-----+
|product_name|year|price|
+------------+----+-----+
|       Nokia|2008| 5000|
|       Nokia|2009| 5000|
|       Apple|2011| 9000|
+------------+----+-----+



[1581. Customer Who Visited but Did Not Make Any Transactions](https://leetcode.com/problems/customer-who-visited-but-did-not-make-any-transactions/?envType=study-plan-v2&envId=top-sql-50)


```sql
select V.customer_id, Count(V.visit_id) as count_no_trans
from Visits as V 
left join Transactions as T
on V.visit_id = T.visit_id
WHERE T.transaction_id IS NULL 
GROUP BY V.customer_id;
```

In [105]:
visits_data = [
    (1, 23),
    (2, 9),
    (4, 30),
    (5, 54),
    (6, 96),
    (7, 54),
    (8, 54)
]
visits_columns = ["visit_id", "customer_id"]
visits_df = spark.createDataFrame(visits_data, visits_columns)

transactions_data = [
    (2, 5, 310),
    (3, 5, 300),
    (9, 5, 200),
    (12, 1, 910),
    (13, 2, 970)
]
transactions_columns = ["transaction_id", "visit_id", "amount"]
transactions_df = spark.createDataFrame(transactions_data, transactions_columns)




In [112]:
result = (
    visits_df
    .join(transactions_df, on="visit_id", how="left")
    .where(transactions_df.transaction_id.isNull())
    .groupBy("customer_id")
    .agg(F.count("visit_id").alias("count_no_trans"))
)

result.show()

+-----------+--------------+
|customer_id|count_no_trans|
+-----------+--------------+
|         54|             2|
|         96|             1|
|         30|             1|
+-----------+--------------+



[197. Rising Temperature](https://leetcode.com/problems/rising-temperature/description/?envType=study-plan-v2&envId=top-sql-50)

```sql
WITH WeatherWithPrevious AS (
    SELECT 
        id,
        recordDate,
        temperature,
        LAG(recordDate) OVER (ORDER BY recordDate) as prev_date,
        LAG(temperature) OVER (ORDER BY recordDate) as prev_temp
    FROM Weather
)
SELECT id
FROM WeatherWithPrevious
WHERE temperature > prev_temp
AND DATEDIFF(day, prev_date, recordDate) = 1;
```

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.window import Window
import pyspark

data = [
    (1, "2015-01-01", 10),
    (2, "2015-01-02", 25),
    (3, "2015-01-03", 20),
    (4, "2015-01-04", 30),
    (5, "2015-01-06", 40) 
]
columns = ["id", "recordDate", "temperature"]

weather_df = spark.createDataFrame(data, columns)


window_spec = Window.orderBy("recordDate")

result_df = (
    weather_df
    .withColumn("prev_date", 
                F.lag("recordDate").over(window_spec))
    .withColumn("prev_temp", 
                F.lag("temperature").over(window_spec))
    .withColumn("days_diff", 
                F.datediff("recordDate", "prev_date"))
    .filter(
        (F.col("temperature") > F.col("prev_temp")) & 
        (F.col("days_diff") == 1)
    )
    .select("id")
)

result_df.show()

[Problem 1661. Average Time of Process per Machine](https://leetcode.com/problems/average-time-of-process-per-machine/description/?envType=study-plan-v2&envId=top-sql-50)

```sql
WITH ProcessTimes AS (
    SELECT 
        machine_id,
        process_id,
        MAX(CASE WHEN activity_type = 'end' THEN timestamp END) - 
        MAX(CASE WHEN activity_type = 'start' THEN timestamp END) as process_time
    FROM Activity
    GROUP BY machine_id, process_id
)
SELECT 
    machine_id,
    ROUND(AVG(process_time), 3) as processing_time
FROM ProcessTimes
GROUP BY machine_id;
```

In [ ]:
from pyspark.sql.types import StructType, StructField, IntegerType, FloatType, StringType

spark = SparkSession.builder.appName("ProcessTimeAnalysis").getOrCreate()

data = [
    (0, 0, "start", 0.712),
    (0, 0, "end", 1.520),
    (0, 1, "start", 3.140),
    (0, 1, "end", 4.120),
    (1, 0, "start", 0.550),
    (1, 0, "end", 1.550),
    (1, 1, "start", 0.430),
    (1, 1, "end", 1.420),
    (2, 0, "start", 4.100),
    (2, 0, "end", 4.512),
    (2, 1, "start", 2.500),
    (2, 1, "end", 5.000)
]

schema = StructType([
    StructField("machine_id", IntegerType(), False),
    StructField("process_id", IntegerType(), False),
    StructField("activity_type", StringType(), False),
    StructField("timestamp", FloatType(), False)
])

activity_df = spark.createDataFrame(data, schema)

In [ ]:
result_df = (
    activity_df
    .groupBy("machine_id", "process_id")
    .agg(
        (F.max(F.when(F.col("activity_type") == "end", F.col("timestamp")).otherwise(None)) - 
        F.max(F.when(F.col("activity_type") == "start", F.col("timestamp")).otherwise(None))).alias("process_time")
    )
    .groupBy("machine_id")
    .agg(
        F.round(F.avg("process_time"), 3).alias("processing_time")
    ).orderBy("machine_id")
)

result_df.show()

[577. Employee Bonus](https://leetcode.com/problems/employee-bonus/description/?envType=study-plan-v2&envId=top-sql-50)

```sql
select [name], bonus
from Employee as E
left join Bonus as B on E.empId = B.empId
where Coalesce(bonus,0) < 1000 
```

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

# Create SparkSession
spark = SparkSession.builder.appName("Employee Bonus Filter").getOrCreate()

# Create Employee DataFrame
data_employee = [
    (3, "Brad", None, 4000),
    (1, "John", 3, 1000),
    (2, "Dan", 3, 2000),
    (4, "Thomas", 3, 4000)
]
columns_employee = ["empId", "name", "supervisor", "salary"]
employee_df = spark.createDataFrame(data_employee, columns_employee)

# Create Bonus DataFrame
data_bonus = [
    (2, 500),
    (4, 2000)
]
columns_bonus = ["empId", "bonus"]
bonus_df = spark.createDataFrame(data_bonus, columns_bonus)

In [ ]:
result_df = (
    employee_df
    .join(bonus_df, on="empId",how="left")
    .filter((F.col("bonus").isNull()) | (F.col("bonus") < 1000))
    .select("name", "bonus")
    
)

result_df.show()

[1280. Students and Examinations](https://leetcode.com/problems/students-and-examinations/description/?envType=study-plan-v2&envId=top-sql-50)

```sql
SELECT
    S.student_id
    ,S.student_name
    ,SU.subject_name
    ,COUNT(E.student_id) attended_exams
FROM Students S
CROSS JOIN Subjects SU
LEFT JOIN Examinations E
    ON E.student_id = S.student_id
    AND SU.subject_name = E.subject_name
GROUP BY S.student_id, S.student_name, SU.subject_name
ORDER BY S.student_id, SU.subject_name
```

In [ ]:
students_data = [
    (1, "Alice"),
    (2, "Bob"),
    (13, "John"),
    (6, "Alex")
]
students_columns = ["student_id", "student_name"]
students_df = spark.createDataFrame(students_data, students_columns)

# Sample data for Subjects table
subjects_data = [
    ("Math",),
    ("Physics",),
    ("Programming",)
]
subjects_columns = ["subject_name"]
subjects_df = spark.createDataFrame(subjects_data, subjects_columns)

# Sample data for Examinations table
examinations_data = [
    (1, "Math"),
    (1, "Physics"),
    (1, "Programming"),
    (2, "Programming"),
    (1, "Physics"),
    (1, "Math"),
    (13, "Math"),
    (13, "Programming"),
    (13, "Physics"),
    (2, "Math"),
    (1, "Math")
]
examinations_columns = ["student_id", "subject_name"]
examinations_df = spark.createDataFrame(examinations_data, examinations_columns)

In [ ]:
base_df = students_df.crossJoin(subjects_df)
print("Base Dataframe",base_df.show())
# Count examinations for each student and subject
exam_counts = (
    examinations_df
    .groupBy("student_id", "subject_name")
    .agg(F.count("*").alias("attended_exams"))
)
print("Exam Counts",exam_counts.show())
# Join the base table with exam counts and fill nulls with 0
result_df = (
    base_df
    .join(exam_counts, 
          ["student_id", "subject_name"], 
          "left")
    .fillna(0, subset=["attended_exams"])
    .orderBy("student_id", "subject_name")
)
print("Result Dataframe",result_df.show())
# Select and order columns to match expected output
final_df = (
    result_df
    .select(
        "student_id",
        "student_name",
        "subject_name",
        F.col("attended_exams")
    )
)
print("Final Dataframe",final_df.show())

[570. Managers with at Least 5 Direct Reports](https://leetcode.com/problems/managers-with-at-least-5-direct-reports/description/?envType=study-plan-v2&envId=top-sql-50)

```sql
SELECT e1.*, e2.*
FROM Employee e1
JOIN Employee e2
ON e1.id = e2.managerId
GROUP BY e1.id, e1.name
HAVING COUNT(e1.id) >= 5;
```

In [ ]:
data = [
    (101, "John", "A", None),
    (102, "Dan", "A", 101),
    (103, "James", "A", 101),
    (104, "Amy", "A", 101),
    (105, "Anne", "A", 101),
    (106, "Ron", "B", 101)
]

schema = ["id", "name", "department", "managerId"]
employee_df = spark.createDataFrame(data, schema)

result_df = employee_df.alias("e1") \
        .join(employee_df.alias("e2"), 
              employee_df.id == employee_df.alias("e2").managerId) \
        .groupBy("e1.id", "e1.name") \
        .agg(F.count("e2.id").alias("direct_reports")) \
        .filter("direct_reports >= 5") \
        .select("name")

result_df.show()

[1934. Confirmation Rate](https://leetcode.com/problems/confirmation-rate/description/?envType=study-plan-v2&envId=top-sql-50)

```sql
SELECT 
    S.user_id, 
    ROUND(
        AVG(
            CASE WHEN c.action = 'confirmed' THEN 1.00 ELSE 0.00 END
            )
        ,2) AS confirmation_rate
FROM 
    Signups AS S
LEFT JOIN Confirmations as C on S.user_id = C.user_id
GROUP BY 
    S.user_id;
```

In [ ]:
signups_data = [
    (3, "2020-03-21 10:16:13"),
    (7, "2020-01-04 13:57:59"),
    (2, "2020-07-29 23:09:44"),
    (6, "2020-12-09 10:39:37"),
]
confirmations_data = [
    (3, "2021-01-06 03:30:46", "timeout"),
    (3, "2021-07-14 14:00:00", "timeout"),
    (7, "2021-06-12 11:57:29", "confirmed"),
    (7, "2021-06-13 12:58:28", "confirmed"),
    (7, "2021-06-14 13:59:27", "confirmed"),
    (2, "2021-01-22 00:00:00", "confirmed"),
    (2, "2021-02-28 23:59:59", "timeout"),
]

# Create DataFrames
signups_df = spark.createDataFrame(signups_data, ["user_id", "time_stamp"])
confirmations_df = spark.createDataFrame(confirmations_data, ["user_id", "time_stamp", "action"])

result = (
    signups_df
    .join(confirmations_df, on="user_id", how="left")
    .withColumn("is_confirmed", F.when(F.col("action") == "confirmed", 1.0).otherwise(0.0))
    .groupBy("user_id")
    .agg(F.round(F.avg("is_confirmed"), 2).alias("confirmation_rate"))
    .fillna({"confirmation_rate": 0.0})  # Users with no requests have a rate of 0
)
# Show result
result.show()

[1174. Immediate Food Delivery II](https://leetcode.com/problems/immediate-food-delivery-ii/description/?envType=study-plan-v2&envId=top-sql-50)

```sql
WITH RankedDelivery AS (
    SELECT 
        customer_id,
        order_date,
        customer_pref_delivery_date,
        ROW_NUMBER() OVER (PARTITION BY customer_id ORDER BY order_date) AS RowRank
    FROM 
        Delivery
)
SELECT 
    ROUND(AVG(CASE WHEN customer_pref_delivery_date = order_date THEN 1.0 ELSE 0.0 END) * 100, 2) AS immediate_percentage
FROM  
    RankedDelivery
WHERE 
    RowRank = 1;

```

[550. Game Play Analysis IV](https://leetcode.com/problems/game-play-analysis-iv/description/?envType=study-plan-v2&envId=top-sql-50)

```sql
WITH FirstLogins AS (
    SELECT 
        player_id,
        MIN(event_date) as first_login_date
    FROM Activity
    GROUP BY player_id
),
ConsecutiveLogins AS (
    SELECT 
        f.player_id,
        CASE 
            WHEN EXISTS (
                SELECT 1 
                FROM Activity a 
                WHERE a.player_id = f.player_id 
                AND a.event_date = DATEADD(day, 1, f.first_login_date)
            ) THEN 1 
            ELSE 0 
        END as logged_consecutive
    FROM FirstLogins f
)
SELECT 
    ROUND(
        CAST(SUM(logged_consecutive) AS FLOAT) / 
        CAST(COUNT(*) AS FLOAT), 
        2
    ) as fraction
FROM ConsecutiveLogins;
```

[Back to Top](#leetcode-sql-problems)